In [2]:
import numpy as np
import pandas as pd

In [3]:
import json
import pydantic


class CeleryConfig(pydantic.BaseModel):
    pass


class SiteConfig(pydantic.BaseModel):
    url: str
    type: str
    routes: list[str]
    confidence: float


class MongoDB(pydantic.BaseModel):
    pass


class Config(pydantic.BaseModel):
    mongodb: MongoDB
    celery: CeleryConfig
    site: list[SiteConfig]


with open("../cluster/configs/celery-beat/config.json", "r") as f:
    sites_cfg = Config.parse_raw(json.dumps(dict(json.load(f))))

sites_cfg_site_routes = {}
for site in sites_cfg.site:
    sites_cfg_site_routes[site.confidence] = set(site.routes)


def valid_or_fake(domain_name: str):
    for confidence, domains in sites_cfg_site_routes.items():
        if domain_name in domains:
            if confidence >= 0.5:
                return True
            else:
                return False

In [4]:
df = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake.gzip")

In [5]:
df["domain"].unique()

array(['monden/', 'it-stiinta/', 'sport/', 'politic/', 'life-death/', '',
       'stiri/economie', 'stiri/externe', 'stiri/actualitate',
       'stiri/actualitate/politica', 'stiri/sport', 'stiri/sci-tech',
       'category/analize/', 'category/news/', 'category/externe/',
       'category/life/', 'category/politica/', 'category/opinii/',
       'category/money/', 'stiri/', 'category/curiozitati-staiti-ca/',
       'category/actualitate/', 'category/stiri-din-romania/',
       'category/mistere-de-pe-mapamond/',
       'category/sanatate-si-medicina/', 'category/stiri-de-pe-mapamond/',
       'category/stirile-zilei/',
       'category/stiri-sanatate-stiri-fitness-stiri-slabire/',
       'category/stiri-internationale/', 'category/timp-liber/',
       'category/stiri-politica/', 'category/utile/', 'covid/'],
      dtype=object)

In [5]:
df

,site,title,domain,content,date
0,https://www.timesnewroman.ro/,Megan Fox e mai bună în dormitor decât în baie,monden/,"Ca să arate că nu este chiar perfectă, Megan F...",2009-10-01 00:00:00+00:00
1,https://www.timesnewroman.ro/,,monden/,"Un coreean a fost arestat pentru că a violat, ...",2009-09-11 00:00:00+00:00
2,https://www.timesnewroman.ro/,Departamentul de scouting de la Radio Zu,monden/,Morar și Buzdugan au împrumutat la Radio Zu mo...,2009-09-18 00:00:00+00:00
3,https://www.timesnewroman.ro/,,monden/,Într-un articol emoționant din ziarul de dumin...,2009-09-14 00:00:00+00:00
4,https://www.timesnewroman.ro/,,monden/,Alina Plugaru mi-a înseninat ziua. Nimic nu cr...,2009-11-01 00:00:00+00:00
...,...,...,...,...,...
78059,https://www.timesnewroman.ro/,"Florina Mihăilă e însărcinată cu oricine, cu t...",monden/,La cît și-o trage nu ar trebui să fie nici o s...,2010-03-08 00:00:00+00:00
78060,https://www.timesnewroman.ro/,Giovanni Becali critică în termeni duri Selecț...,monden/,Nu toată lumea e mulțumită că Paula Seling și ...,2010-03-07 00:00:00+00:00
78061,https://www.timesnewroman.ro/,Sexy Brăileanca o să îmbrace hainele sponsorul...,monden/,Actrița porno Sexy Brăileanca a devenit ieri i...,2010-03-04 00:00:00+00:00
78062,https://www.timesnewroman.ro/,"Femeia ta gonflabilă consumă ""Activia""?",monden/,Ai o femeie gonflabilă în care ai pompat cam m...,2010-03-04 00:00:00+00:00


In [35]:
correspondings = {
    ("it-stiinta/", "covid/", "category/sanatate-si-medicina/", "category/analize/"): {
        "stiri/sci-tech"
    },
    ("sport/", None): {"stiri/sport"},
    ("category/money/", None): {"stiri/economie"},
    ("politic/", "category/politica/", "category/stiri-politica/"): {"stiri/actualitate/politica"},
    ("life-death/", "category/externe/", "category/stiri-internationale/", "category/life/"): {
        "stiri/externe"
    },
    (
        "monden/",
        "category/news/",
        "stiri/",
        "category/stirile-zilei/",
        "category/actualitate/",
    ): {"stiri/actualitate"},
    (
        "category/opinii/",
        "category/curiozitati-staiti-ca/",
        "category/stiri-din-romania/",
        "category/mistere-de-pe-mapamond/",
        "category/stiri-de-pe-mapamond/",
        "category/stiri-sanatate-stiri-fitness-stiri-slabire/",
        "category/timp-liber/",
        "category/utile/",
    ): {"", None},
}


In [ ]:
news = []
for fake_domain in correspondings:
    for year in pd.date_range(
        start=min(df["date"]),
        end=max(df["date"]) + pd.Timedelta(days=366),
        freq="y",
        tz="UTC",
    ):
        domains = [
            sum(
                [
                    len(
                        df[
                            (df.date >= year)
                            & (df.date < year + pd.Timedelta(days=366))
                            & (df.domain == domain)
                        ]
                    )
                    for domain in fake_domain
                ]
            )
        ]
        domains.append(
            sum(
                [
                    len(
                        df[
                            (df.date >= year)
                            & (df.date < year + pd.Timedelta(days=366))
                            & (df.domain == domain)
                        ]
                    )
                    for domain in correspondings[fake_domain]
                ]
            )
        )
        print(domains)
        min_length = min(domains)
        news.append({"year": year, "min": min_length})
        print(f"{fake_domain}-{correspondings[fake_domain]} : {min_length}")


In [112]:
df_balanced = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])


In [113]:
df_fake = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])
df_valid = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])
for fake_domain in correspondings:
    for domain in fake_domain:
        df_fake = pd.concat(
            [
                df_fake,
                df[(df.domain == domain)],
            ]
        )
    for domain in correspondings[fake_domain]:
        df_valid = pd.concat(
            [
                df_valid,
                df[(df.domain == domain)],
            ]
        )
df_fake = df_fake.sample(frac=1).reset_index(drop=True)
df_valid = df_valid.sample(frac=1).reset_index(drop=True)
for new in news:
    if new["min"] != 0:
        df_balanced = pd.concat(
            [
                df_balanced,
                df_valid[
                    (df_valid.date >= pd.Timestamp(new["year"].to_datetime64(), tz="UTC"))
                    & (
                        df_valid.date
                        < pd.Timestamp(new["year"].to_datetime64(), tz="UTC")
                        + pd.Timedelta(days=366)
                    )
                ][: new["min"]],
                df_fake[
                    (df_fake.date >= pd.Timestamp(new["year"].to_datetime64(), tz="UTC"))
                    & (
                        df_fake.date
                        < pd.Timestamp(new["year"].to_datetime64(), tz="UTC")
                        + pd.Timedelta(days=366)
                    )
                ][: new["min"]],
            ]
        )


In [ ]:
df_balanced.to_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")

In [6]:
import json
import pydantic


class CeleryConfig(pydantic.BaseModel):
    pass


class SiteConfig(pydantic.BaseModel):
    url: str
    type: str
    routes: list[str]
    confidence: float


class MongoDB(pydantic.BaseModel):
    pass


class Config(pydantic.BaseModel):
    mongodb: MongoDB
    celery: CeleryConfig
    site: list[SiteConfig]


with open("../cluster/configs/celery-beat/config.json", "r") as f:
    sites_cfg = Config.parse_raw(json.dumps(dict(json.load(f))))

routes_confidence = {}
for site in sites_cfg.site:
    routes_confidence[site.url] = site.confidence

In [8]:
import pandas as pd
df_balanced = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")
df_balanced["label"] = [routes_confidence[site] for site in df_balanced["site"]]

In [10]:
df_balanced.to_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")